In [ ]:
!pip install langchain_community

In [ ]:
!pip3 install pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
#from langchain_core.text_splitter import RecursiveCharacterTextSplitter

#loader = PyPDFLoader("keringra2023.pdf")
loader = PyPDFLoader("deu-2023-vf_vdef.pdf", extract_images=False)
pages = loader.load_and_split()
pages

starting_page = 1
ending_page = 165

# Extract pages WHERE "page" key of pages' "metadata" are between starting_page and ending_page
pages_RSE_pypdf = [page for page in pages if page.metadata.get("page") in range(starting_page+1, ending_page+2)]
pages_RSE_pypdf = pages

# print first page of pages_RSE
print(pages_RSE_pypdf[-1])



In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create text_splitter from RecursiveCharacterTextSplitter with French separators
#separators = [".", "!", "?", ":", ";", "\n\n"]
separators = ["\n\n"]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=80, separators=separators)
mySplittedDocs = text_splitter.split_documents(pages_RSE_pypdf)

for doc in mySplittedDocs:
    # Trim leading unwanted characters (., !, ?) from doc.page_content
    while doc.page_content and doc.page_content[0] in separators:
        doc.page_content = doc.page_content[1:]
    
    # Clean line breaks in doc.page_content
    doc.page_content = doc.page_content.replace("\n", " ").strip()
    # Remove extra whitespaces
    doc.page_content = ' '.join(doc.page_content.split())
print(mySplittedDocs[88], "\n")
print(mySplittedDocs[89], "\n")
print(mySplittedDocs[90], "\n")

In [ ]:
#!pip install llama-index
#%conda install -c conda-forge sentence-transformers

In [3]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Read the pdf file with SimpleDirectoryReader
reader = SimpleDirectoryReader(input_files=["deu-2023-vf_vdef.pdf"]).load_data()
text_splitter = SentenceSplitter(chunk_size=350, chunk_overlap=80, include_metadata=True)
mySplittedDocs = text_splitter.get_nodes_from_documents(reader)

In [2]:
print(mySplittedDocs[88].metadata)
print(mySplittedDocs[88])

# modify the metadata of node 88, creation_date = 2023-01-01
#mySplittedDocs[88].metadata["creation_date"] = "2023-01-01"
#print(mySplittedDocs[88].metadata)

{'page_label': '20', 'file_name': 'deu-2023-vf_vdef.pdf', 'file_path': 'deu-2023-vf_vdef.pdf', 'file_type': 'application/pdf', 'file_size': 5539209, 'creation_date': '2024-05-05', 'last_modified_date': '2024-05-05'}
Node ID: f7bee576-7c20-4564-b8c1-4e41f1c6fb8e
Text: L’activité des Perfumes Loewe est  intégrée au sein des Parfums
et Cosmétiques. Marc Jacobs , créée à New York en 1984, est la marque
éponyme  de son fondateur, et fait partie de LVMH depuis 1997. Elle se
veut, à travers ses collections de prêt‑à‑porter homme et femme,  de
maroquinerie et de souliers, le symbole d’une mode urbaine  et
irrévéren...


In [5]:
def process_nodes(nodes, reporting_year):
    documents = []
    metadatas = []
    ids = []
    
    for i, node in enumerate(nodes):
        id = str(i)
        document = node.text
        metadata = node.metadata
        metadata["reporting_year"] = reporting_year
        ids.append(id)
        documents.append(document)
        metadatas.append(metadata)
    
    return documents, metadatas, ids

n2docs, n2metas, n2ids = process_nodes(mySplittedDocs, "2023")
print(n2docs[88])
print(n2metas[88])
print(n2ids[88])

L’activité des Perfumes Loewe est 
intégrée au sein des Parfums et Cosmétiques.
Marc Jacobs , créée à New York en 1984, est la marque éponyme 
de son fondateur, et fait partie de LVMH depuis 1997. Elle se 
veut, à travers ses collections de prêt‑à‑porter homme et femme, 
de maroquinerie et de souliers, le symbole d’une mode urbaine 
et irrévérencieuse, culturelle mais aussi engagée.
Celine , fondée en 1945 par Céline Vipiana, et détenue par LVMH 
depuis 1996, développe une offre féminine et masculine de 
prêt‑à‑porter, maroquinerie, souliers, accessoires et fragrances 
féminins et masculins.Kenzo , créée en 1970, a rejoint le Groupe en 1993. Célèbre pour 
ses imprimés foisonnants et ses couleurs vibrantes, la Maison 
exerce ses activités dans le prêt‑à‑porter féminin et masculin, 
les accessoires de mode, les souliers et la maroquinerie. Son 
activité dans le domaine des parfums est intégrée dans le groupe 
d’activités Parfums et Cosmétiques.
{'page_label': '20', 'file_name': 'deu-2023

In [ ]:
%pip install llama-index-embeddings-huggingface

In [4]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of CUDA devices available
    num_devices = torch.cuda.device_count()
    print(f"Number of CUDA devices available: {num_devices}")
    
    # List each device and its corresponding name
    for i in range(num_devices):
        print(f"CUDA Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. Listing devices is not possible.")

Number of CUDA devices available: 1
CUDA Device 0: NVIDIA GeForce GTX 1050


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Use sentence transformers as embedding model for Chroma with all-MiniLM-L6-v2
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model_name = "dangvantuan/sentence-camembert-base"
#embedding_model = SentenceTransformer(embedding_model_name)
embedding_model = HuggingFaceEmbedding(model_name=embedding_model_name, device="cuda:0" if torch.cuda.is_available() else "cpu")

index2 = VectorStoreIndex(
        mySplittedDocs,
        embed_model=embedding_model
    )


In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
retriever = VectorIndexRetriever(
        index=index2,
        similarity_top_k=8,
    )

retrieved_nodes2 = retriever.retrieve("Emissions Scope 2 tonnes CO2e")
print(retrieved_nodes2[0].text)

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=8,
    )

retrieved_nodes = retriever.retrieve("Emissions Scope 2 tonnes CO2e")
print(retrieved_nodes[0])

In [ ]:
print(retrieved_nodes[0].text)

In [ ]:
retriever_splitDocs = vectorstore_splittedDocs.as_retriever()
retriever_splitDocs.invoke("zones clés pour la biodiversité")

# Maximizing vectorizer with CUDA from https://github.com/johnnycode8/chromadb_quickstart/blob/main/chroma_fast_vectorize.py

In [7]:
import chromadb
from chromadb.utils import embedding_functions
import multiprocessing as mp
import time
#from tqdm.autonotebook import tqdm, trange
from tqdm import tqdm

def producer_from_nodes(nodes, reporting_year, queue, batch_size):
    documents, metadatas, ids = process_nodes(nodes, reporting_year)
    
    # Splitting the data into batches and putting them into the queue
    for i in range(0, len(ids), batch_size):
        batch_documents = documents[i:i+batch_size]
        batch_metadatas = metadatas[i:i+batch_size]
        batch_ids = ids[i:i+batch_size]
        queue.put((batch_documents, batch_metadatas, batch_ids))

    # Signal the end of the queue
    queue.put(None)

def consumer(use_cuda, queue):
    chroma_client = chromadb.PersistentClient(path="my_vectordb")
    device = 'cuda:0' if use_cuda else 'cpu'
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/all-MiniLM-L6-v2", device=device)
    collection = chroma_client.get_collection(name="my_documents", embedding_function=sentence_transformer_ef)

    while True:
        batch = queue.get()
        if batch is None:
            break
        collection.add(documents=batch[0], metadatas=batch[1], ids=batch[2])

if __name__ == "__main__":
    chroma_client = chromadb.PersistentClient(path="my_vectordb")
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/all-MiniLM-L6-v2")

    try:
        chroma_client.get_collection(name="my_documents")
        chroma_client.delete_collection(name="my_documents")
    except Exception as err:
        print(err)

    collection = chroma_client.create_collection(name="my_documents", embedding_function=sentence_transformer_ef)

    queue = mp.Queue()

    producer_process = mp.Process(target=producer_from_nodes, args=(mySplittedDocs, "2023", queue, 10000,))
    consumer_process = mp.Process(target=consumer, args=(True, queue,))

    start_time = time.time()

    producer_process.start()
    consumer_process.start()

    producer_process.join()
    consumer_process.join()

    print(f"Elapsed seconds: {time.time() - start_time:.0f} Record count: {collection.count()}")

# Batch custom function

In [1]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import torch
from langchain_core.documents import Document


# Read the pdf file with SimpleDirectoryReader
reader = SimpleDirectoryReader(input_files=["deu-2023-vf_vdef.pdf"]).load_data()
text_splitter = SentenceSplitter(chunk_size=350, chunk_overlap=80, include_metadata=True)
mySplittedDocs = text_splitter.get_nodes_from_documents(reader)

def process_nodes(nodes, reporting_year, company_name):
    documents = []
    metadatas = []
    ids = []
    
    for i, node in enumerate(nodes):
        id = str(i)
        document = node.text
        metadata = node.metadata
        metadata["reporting_year"] = reporting_year
        metadata["company_name"] = company_name
        ids.append(id)
        documents.append(document)
        metadatas.append(metadata)
    
    return documents, metadatas, ids

n2docs, n2metas, n2ids = process_nodes(mySplittedDocs, reporting_year="2023", company_name="LVMH")
print(n2docs[88])
print(n2metas[88])
print(n2ids[88])

# Remove the line breaks in n2docs
n2docs = [doc.replace("\n", " ").strip() for doc in n2docs]

# Assuming n2docs, n2metas, and n2ids are already defined as a result of process_nodes(mySplittedDocs, "2023")
documents = [Document(page_content=doc, metadata=meta) for doc, meta in zip(n2docs, n2metas)]

# Use sentence transformers as embedding model for Chroma with all-MiniLM-L6-v2
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
#embedding_model_name = "dangvantuan/sentence-camembert-base"

embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name, model_kwargs={"device": "cuda:0" if torch.cuda.is_available() else "cpu"})


vectorstore_splittedDocs = Chroma.from_documents(documents, embedding=embedding_model)


L’activité des Perfumes Loewe est 
intégrée au sein des Parfums et Cosmétiques.
Marc Jacobs , créée à New York en 1984, est la marque éponyme 
de son fondateur, et fait partie de LVMH depuis 1997. Elle se 
veut, à travers ses collections de prêt‑à‑porter homme et femme, 
de maroquinerie et de souliers, le symbole d’une mode urbaine 
et irrévérencieuse, culturelle mais aussi engagée.
Celine , fondée en 1945 par Céline Vipiana, et détenue par LVMH 
depuis 1996, développe une offre féminine et masculine de 
prêt‑à‑porter, maroquinerie, souliers, accessoires et fragrances 
féminins et masculins.Kenzo , créée en 1970, a rejoint le Groupe en 1993. Célèbre pour 
ses imprimés foisonnants et ses couleurs vibrantes, la Maison 
exerce ses activités dans le prêt‑à‑porter féminin et masculin, 
les accessoires de mode, les souliers et la maroquinerie. Son 
activité dans le domaine des parfums est intégrée dans le groupe 
d’activités Parfums et Cosmétiques.
{'page_label': '20', 'file_name': 'deu-2023

In [23]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import torch
from langchain_core.documents import Document

def create_vectorstore_from_pdf(input_file_path, reporting_year="2023", company_name="LVMH", embedding_model_name="sentence-transformers/all-MiniLM-L6-v2"):
    # Read the pdf file with SimpleDirectoryReader
    reader = SimpleDirectoryReader(input_files=[input_file_path]).load_data()
    text_splitter = SentenceSplitter(chunk_size=350, chunk_overlap=80, include_metadata=True)
    splitted_docs = text_splitter.get_nodes_from_documents(reader)

    def process_nodes(nodes):
        documents = []
        metadatas = []
        ids = []
        
        for i, node in enumerate(nodes):
            id = str(i)
            document = node.text
            metadata = node.metadata
            metadata["reporting_year"] = reporting_year
            metadata["company_name"] = company_name
            ids.append(id)
            documents.append(document)
            metadatas.append(metadata)
        
        return documents, metadatas, ids

    n2docs, n2metas, _ = process_nodes(splitted_docs)

    # Remove the line breaks in n2docs
    n2docs = [doc.replace("\n", " ").strip() for doc in n2docs]

    documents = [Document(page_content=doc, metadata=meta) for doc, meta in zip(n2docs, n2metas)]

    embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name, model_kwargs={"device": "cuda:0" if torch.cuda.is_available() else "cpu"})

    vectorstore_splittedDocs = Chroma.from_documents(documents, embedding=embedding_model)
    
    return vectorstore_splittedDocs

# Apply the function to the pdf file
vectorstore = create_vectorstore_from_pdf("deu-2023-vf_vdef.pdf", reporting_year="2023", company_name="LVMH", embedding_model_name="sentence-transformers/all-MiniLM-L6-v2")

In [30]:
# Query the vectorstore
results = vectorstore.as_retriever(kwargs={'k':8}).invoke("Emissions Scope 2 tonnes CO2e")
for result in results:
    print(result.page_content)
    print("\n")
    print(result.metadata)
    print("\n\n\n")

En 2022, les émissions totales de l’empreinte s’élevaient à 6,4 millions de tonnes eqCO2 dont 6,1 millions de tonnes pour le scope 3   qui se décomposent de la manière suivante : Catégories du GHG Protocol Quantité de gaz à effet de serre  (en milliers de tonnes équivalent CO2)  Achats de produits et services dont : 3 370 —  Laines et laines précieuses 749 —  C u i r s 377 —  C o t o n 245 —  O r 520 —  V e r r e 143 —  Raisins et alcool de raisins 112 Immobilisations 1 535 Énergie (hors scopes 1 et 2) 95 Transport amont et distribution 576 Déchets générés 9 Déplacements professionnels 69 Trajets domicile – lieu de travail 171 Utilisation des produits vendus 133 Fin de vie des produits vendus 41 Investissements 137 Total 6 135


{'company_name': 'LVMH', 'creation_date': '2024-05-05', 'file_name': 'deu-2023-vf_vdef.pdf', 'file_path': 'deu-2023-vf_vdef.pdf', 'file_size': 5539209, 'file_type': 'application/pdf', 'last_modified_date': '2024-05-05', 'page_label': '105', 'reporting_year': '2

In [28]:
docs = vectorstore.similarity_search("Emissions Scope 2 tonnes CO2e")
for doc in docs:
    print(doc.page_content)
    print("\n")
    print(doc.metadata)
    print("\n\n\n")

En 2022, les émissions totales de l’empreinte s’élevaient à 6,4 millions de tonnes eqCO2 dont 6,1 millions de tonnes pour le scope 3   qui se décomposent de la manière suivante : Catégories du GHG Protocol Quantité de gaz à effet de serre  (en milliers de tonnes équivalent CO2)  Achats de produits et services dont : 3 370 —  Laines et laines précieuses 749 —  C u i r s 377 —  C o t o n 245 —  O r 520 —  V e r r e 143 —  Raisins et alcool de raisins 112 Immobilisations 1 535 Énergie (hors scopes 1 et 2) 95 Transport amont et distribution 576 Déchets générés 9 Déplacements professionnels 69 Trajets domicile – lieu de travail 171 Utilisation des produits vendus 133 Fin de vie des produits vendus 41 Investissements 137 Total 6 135


{'company_name': 'LVMH', 'creation_date': '2024-05-05', 'file_name': 'deu-2023-vf_vdef.pdf', 'file_path': 'deu-2023-vf_vdef.pdf', 'file_size': 5539209, 'file_type': 'application/pdf', 'last_modified_date': '2024-05-05', 'page_label': '105', 'reporting_year': '2

# Functions to add docs



In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import torch
from langchain_core.documents import Document
import html
import unicodedata

def decode_unicode(text):
    # Manually replace known pseudo-escape sequences
    replacements = {
        '/uni00A0': '\u00A0',  # Non-breaking space
        '/uni2011': '\u2011',  # Non-breaking hyphen
        # Add more replacements as needed
    }
    for pseudo, char in replacements.items():
        text = text.replace(pseudo, char)
    
    # Decode HTML entities and normalize
    decoded_text = html.unescape(text)
    decoded_text = unicodedata.normalize('NFKC', decoded_text)
    
    # Remove combining characters
    decoded_text = ''.join(c for c in decoded_text if not unicodedata.combining(c))
    return decoded_text

def pdf_to_docs(input_file_path, reporting_year="2023", company_name="LVMH", embedding_model_name="sentence-transformers/all-MiniLM-L6-v2"):
    # Read the pdf file with SimpleDirectoryReader
    reader = SimpleDirectoryReader(input_files=[input_file_path]).load_data()
    text_splitter = SentenceSplitter(chunk_size=350, chunk_overlap=80, include_metadata=True)
    splitted_docs = text_splitter.get_nodes_from_documents(reader)

    def process_nodes(nodes):
        documents = []
        metadatas = []
        ids = []
        
        for i, node in enumerate(nodes):
            id = str(i)
            #document = node.text
            document = decode_unicode(node.text)  # Apply decode_unicode() here

            metadata = node.metadata
            metadata["reporting_year"] = reporting_year
            metadata["company_name"] = company_name
            ids.append(id)
            documents.append(document)
            metadatas.append(metadata)
        
        return documents, metadatas, ids

    n2docs, n2metas, _ = process_nodes(splitted_docs) # removed ids to let Chroma create hashes instead as ids

    # Remove the line breaks in n2docs
    n2docs = [doc.replace("\n", " ").strip() for doc in n2docs]

    documents = [Document(page_content=doc, metadata=meta) for doc, meta in zip(n2docs, n2metas)]
    
    return documents
embedding_model_name="sentence-transformers/all-MiniLM-L6-v2"
chroma_directory = 'db/'
embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name, model_kwargs={"device": "cuda:0" if torch.cuda.is_available() else "cpu"})

db = Chroma(persist_directory=chroma_directory, embedding_function=embedding_model)
docs2023 = pdf_to_docs("deu-2023-vf_vdef.pdf", reporting_year="2023", company_name="LVMH", embedding_model_name=embedding_model_name)

db.add_documents(documents=docs2023)
print(f"DB Count: {db._collection.count()}")
docs2022 = pdf_to_docs("240326_hermes_urd2023_fr.pdf",
                       reporting_year="2023", company_name="Hermes",
                       embedding_model_name=embedding_model_name)
db.add_documents(documents=docs2022)


# BATCH process all pdfs

In [2]:
import os
import re

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import torch
from langchain_core.documents import Document
import html
import unicodedata

def decode_unicode(text):
    # Manually replace known pseudo-escape sequences
    replacements = {
        '/uni00A0': '\u00A0',  # Non-breaking space
        '/uni2011': '\u2011',  # Non-breaking hyphen
        # Add more replacements as needed
    }
    for pseudo, char in replacements.items():
        text = text.replace(pseudo, char)
    
    # Decode HTML entities and normalize
    decoded_text = html.unescape(text)
    decoded_text = unicodedata.normalize('NFKC', decoded_text)
    
    # Remove combining characters
    decoded_text = ''.join(c for c in decoded_text if not unicodedata.combining(c))
    return decoded_text

def pdf_to_docs(input_file_path, reporting_year, company_name):
    # Read the pdf file with SimpleDirectoryReader
    reader = SimpleDirectoryReader(input_files=[input_file_path]).load_data()
    text_splitter = SentenceSplitter(chunk_size=350, chunk_overlap=80, include_metadata=True)
    splitted_docs = text_splitter.get_nodes_from_documents(reader)

    def process_nodes(nodes):
        documents = []
        metadatas = []
        ids = []
        
        for i, node in enumerate(nodes):
            id = str(i)
            #document = node.text
            document = decode_unicode(node.text)  # Apply decode_unicode() here

            metadata = node.metadata
            metadata["reporting_year"] = reporting_year
            metadata["company_name"] = company_name
            ids.append(id)
            documents.append(document)
            metadatas.append(metadata)
        
        return documents, metadatas, ids

    n2docs, n2metas, _ = process_nodes(splitted_docs)

    # Remove the line breaks in n2docs
    n2docs = [doc.replace("\n", " ").strip() for doc in n2docs]

    documents = [Document(page_content=doc, metadata=meta) for doc, meta in zip(n2docs, n2metas)]
    
    return documents

embedding_model_name = "dangvantuan/sentence-camembert-base"
#embedding_model_name="sentence-transformers/all-MiniLM-L6-v2" # Uncomment this line for x4 faster embeddings

chroma_directory = 'dbRSEcamemBERT/'
embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name, model_kwargs={"device": "cuda:0" if torch.cuda.is_available() else "cpu"})

db = Chroma(persist_directory=chroma_directory, embedding_function=embedding_model)

def process_all_pdfs(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".pdf"):
                print(f"Processing file {file}")
                # Extract COMPANY_NAME and REPORTING_YEAR from the path or filename
                match = re.search(r"([^\\]+)_(\d{4})\.pdf$", file)
                if match:
                    company_name = match.group(1)
                    reporting_year = match.group(2)
                    file_path = os.path.join(root, file)
                    try:
                        # Apply pdf_to_docs to the identified PDF file
                        docs_from_pdf = pdf_to_docs(file_path, reporting_year=reporting_year, company_name=company_name)
                        # Here, you can add the documents to your database or perform further processing
                        print(f"Docs successfully created from {file_path}.")
                        print(f"The DB's data count increased from {db._collection.count()}", end=" ")

                        # Add docs to the vector db
                        db.add_documents(documents=docs_from_pdf)
                        print(f" -to-> {db._collection.count()}")
                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")

# Example usage
root_folder = "../Corpus/RAG_RSE_Corpus"
process_all_pdfs(root_folder)

Processing file BarbaraBui_2017.pdf
Docs successfully created from ../Corpus/RAG_RSE_Corpus\BarabaraBui\BarbaraBui_2017.pdf.
The DB's data count increased from 0  -to-> 351
Processing file BarbaraBui_2018.pdf
Docs successfully created from ../Corpus/RAG_RSE_Corpus\BarabaraBui\BarbaraBui_2018.pdf.
The DB's data count increased from 351  -to-> 702
Processing file BarbaraBui_2019.pdf
Docs successfully created from ../Corpus/RAG_RSE_Corpus\BarabaraBui\BarbaraBui_2019.pdf.
The DB's data count increased from 702  -to-> 1086
Processing file BarbaraBui_2020.pdf
Docs successfully created from ../Corpus/RAG_RSE_Corpus\BarabaraBui\BarbaraBui_2020.pdf.
The DB's data count increased from 1086  -to-> 1614
Processing file BarbaraBui_2021.pdf
Docs successfully created from ../Corpus/RAG_RSE_Corpus\BarabaraBui\BarbaraBui_2021.pdf.
The DB's data count increased from 1614  -to-> 2001
Processing file BarbaraBui_2022.pdf
Docs successfully created from ../Corpus/RAG_RSE_Corpus\BarabaraBui\BarbaraBui_2022.pd

In [3]:
# Export list of only ids from Chroma db collection
ids_only_result = db.get(include=[])
print(ids_only_result['ids'])

['00025a2f-b32b-4922-8804-a39858539d3e', '00025d73-2697-4bae-9e7c-3a52102e63b0', '000436fe-2d9b-4be8-8672-7387d4a966ff', '00058818-c88a-4844-a02d-0c99863f76ed', '0006a74a-ef90-4471-9ea8-048c412a1bdc', '0006fec0-a58a-4577-b064-e17e4e098236', '0007051b-cdca-48e1-8170-a16b39182697', '00082ec9-46b6-45bd-b6c2-8c8a6a897678', '0009075e-d024-4435-bb54-f3d6eaa7a373', '000be7db-3bcd-4428-adf7-0b80fa1a3604', '000f7ca5-b34b-4a7f-981a-c300ea710f18', '000ffbe0-a740-43f0-8b21-98053c75228b', '00104567-cf2d-49f0-a8f1-72e323469f59', '0010a235-f0c0-416f-9a2b-ea6578325e24', '0010b3fb-31d5-4cc5-863c-f4d62f027851', '00115b5f-4f35-4559-a952-bfcde86f82dc', '00163859-bc1e-4b97-b7c3-2352cfce1404', '0018dacb-f630-4d1e-87b6-fe7858559276', '001a116a-4183-42fc-8ec2-7df5e182e60a', '001c0650-bdd6-4bc3-afee-2df7a715b608', '001c0d22-3b91-4e31-a22b-fefacad61781', '001c6ec0-80f8-4b9a-9b7a-3712e0392cd5', '001d4b23-86df-4371-855e-40db25632706', '001d5a92-a7c0-42ca-9d05-7cd8d1a0cac6', '001e708e-6482-45eb-aba4-481866d8257d',

In [8]:
test_id = 'ffc8d9ad-44a4-461b-a2e6-bd93dfae1451'
# pring get doc from db
print(db._collection.get(ids=test_id)['documents'][0])

La Fondation accompagne  et finance ainsi une promotion de 12 projets théâtre dans le  cadre scolaire, et invite l’un d’ entre eux, le lauréat, à présenter son  spectacle sur la scène de l’Odéon à l’issue des 2 ans. En mai 2020, le  confinement a empêché le lauréat, une école primaire de Marseille,  de jouer à l’Odéon ; les 100 écoliers concernés ont cependant réalisé  un livre de 144 pages qui présente leurs 2 années de travail sur Les  Fables de La Fontaine . ●●“Tous en scène” , qui permet de financer des projets théâtre hors  du cadre scolaire, en impliquant les collaborateurs volontaires  du Groupe. Cette opération annuelle solidaire nationale est organisée par la  Fondation Casino avec l’ appui des enseignes du Groupe dans les  magasins Casino, Franprix, et Cdiscount. L ’ édition 2020 a permis de  récolter presque 100 000 euros pour les deux associations partenaires  de la Fondation “ Apprentis d’ Auteuil” et “L ’Envol” afin de développer  de nouveaux projets théâtre au profit des